# hyperparameterTunningResults4
Andrew E. Davidson  
aedavids@ucsc.edu  
1/3/23

see extraCellularRNA/deconvolutionAnalysis/doc/bestCurratedNotes.md for a discussion about tunning 


In [1]:
import ipynbname

# use display() to print an html version of a data frame
# useful if dataFrame output is not generated by last like of cell
from IPython.display import display

import numpy as np
import pandas as pd
# display all columns
pd.set_option('display.max_columns', None)

import pathlib as pl
import pprint as pp
import os
import sys

In [2]:
# setting the python path allows us to run python scripts from using
# the CLI. 
ORIG_PYTHONPATH = os.environ['PYTHONPATH']

notebookPath = ipynbname.path()
deconvolutionModules = notebookPath.parent.joinpath("../../python")
print("deconvolutionModules: {}\n".format(deconvolutionModules))

PYTHONPATH = ORIG_PYTHONPATH + f':{deconvolutionModules}'
print("PYTHONPATH: {}\n".format(PYTHONPATH))

os.environ["PYTHONPATH"] = PYTHONPATH
PYTHONPATH = os.environ["PYTHONPATH"]
print("PYTHONPATH: {}\n".format(PYTHONPATH))

# to be able to import our local python files we need to set the sys.path
# https://stackoverflow.com/a/50155834
sys.path.append( str(deconvolutionModules) )
print("\nsys.path:\n{}\n".format(sys.path))

deconvolutionModules: /private/home/aedavids/extraCellularRNA/deconvolutionAnalysis/jupyterNotebooks/hyperParameterTunning/../../python

PYTHONPATH: :/private/home/aedavids/extraCellularRNA/src:/private/home/aedavids/extraCellularRNA/deconvolutionAnalysis/jupyterNotebooks/hyperParameterTunning/../../python

PYTHONPATH: :/private/home/aedavids/extraCellularRNA/src:/private/home/aedavids/extraCellularRNA/deconvolutionAnalysis/jupyterNotebooks/hyperParameterTunning/../../python


sys.path:
['/private/home/aedavids/extraCellularRNA/deconvolutionAnalysis/jupyterNotebooks/hyperParameterTunning', '/private/home/aedavids/extraCellularRNA/deconvolutionAnalysis/jupyterNotebooks/hyperParameterTunning', '/private/home/aedavids/extraCellularRNA/src', '/private/home/aedavids/miniconda3/envs/extraCellularRNA/lib/python311.zip', '/private/home/aedavids/miniconda3/envs/extraCellularRNA/lib/python3.11', '/private/home/aedavids/miniconda3/envs/extraCellularRNA/lib/python3.11/lib-dynload', '', '/private/h

In [3]:
from analysis.hyperParameterTunningMetrics import metricsRunner, elifeCols, lungCols
from analysis.hyperParameterTunningMetrics import findSummaryMetricsCols
from analysis.utilities import findAllCategories, findAllGenes
from analysis.utilities import findIntersectionsWithDegree
from analysis.utilities import loadDictionary

In [4]:
root = "/private/groups/kimlab/aedavids/deconvolution/1vsAll-~gender_category"
notebookName = ipynbname.name()
outDir = f'{root}/hyperParameter/{notebookName}.out'
print( f'output dir: \n{outDir}' )
os.makedirs(outDir, exist_ok=True)

output dir: 
/private/groups/kimlab/aedavids/deconvolution/1vsAll-~gender_category/hyperParameter/hyperparameterTunningResults4.out


# stage 2: aedwip find all degree1

**"bestNFindAllDegree1_wlN** is not a great algorithm. The only usefully bit is we can mine the interesection dictionary for degree 1 genes.

Expect best25GTEx_TCGA and best20FindAllDegree1_wl5 to have same degree1. Best25 should have more genes. actually did a little better. 

b20FindAll has 28 more genes than best20 and 22 less genes then best25.

b20FindAll has 8 more categories with degree1 genes than b20 and has 2 more degree1 categories and b25

best20 and b20Findall have the same number of categories. this is 2 more than best25!

In [5]:
def evaluate(
    root: str,
    outDir: str,    
    resultsDirs : list[str],
    outFilePrefix : str,
    metric : str,
    stageName : str,
    threshold : float,
    verbose : bool = False,
    ) -> tuple[pd.DataFrame, pd.DataFrame] :

    retDF, retBellowThresholdDF = metricsRunner(root, outDir, outFilePrefix, resultsDirs, 
                           metric=metric, threshold=threshold, verbose=verbose)

    display( retDF.loc[:, findSummaryMetricsCols(metric) + elifeCols  ] )

    print(f'\n {stageName} classes < {threshold} {metric}')
    selectRowsBellow = retBellowThresholdDF.loc[:, "stage"] == stageName
    
    display( retBellowThresholdDF.loc[selectRowsBellow, 'category'] )

    return (retDF, retBellowThresholdDF)

In [6]:
def evaluateBestResults():
    bestFindAllDegree1_wlResultsDirs = [    
        # "best20GTEx_TCGA",     
        # "best25GTEx_TCGA",
        "best20FindAllDegree1_wl5",    
        "best500FindAllDegree1_wl500",
              
    ]
    
    threshold=0.7
    # stageName =  "best20FindAllDegree1_wl5"
    stageName =  "best500FindAllDegree1_wl500"
    bestMetricsAllDegree1DF, bestFindAllDegree1_wlResultsDirs = evaluate(
                                                        root=root,
                                                        outDir=outDir,
                                                        resultsDirs=bestFindAllDegree1_wlResultsDirs, 
                                                        outFilePrefix=stageName,
                                                        metric='sensitivity',
                                                        stageName=stageName,
                                                        threshold=threshold,
                                                        )
    
    display( bestMetricsAllDegree1DF.loc[:, findSummaryMetricsCols('sensitivity') + elifeCols  ] )

    return(bestMetricsAllDegree1DF, bestFindAllDegree1_wlResultsDirs)

bestMetricsAllDegree1DF, bestAllBellowThresholdDF = evaluateBestResults()


saving : /private/groups/kimlab/aedavids/deconvolution/1vsAll-~gender_category/hyperParameter/hyperparameterTunningResults4.out/best500FindAllDegree1_wl500.sensitivity.0.7.csv

saving : /private/groups/kimlab/aedavids/deconvolution/1vsAll-~gender_category/hyperParameter/hyperparameterTunningResults4.out/best500FindAllDegree1_wl500.sensitivity.bellow.0.7.csv


id,mean_sensitivity,std_sensitivity,numGenes,numTypes,numDegree1,numAboveThreshold,LUAD,LUSC,COAD,READ,ESCA,LIHC,STAD,Whole_Blood
best20FindAllDegree1_wl5,0.708614,0.216666,313,83,46,45,0.443,0.502,0.456,0.786,0.162,0.735,0.280,0.987
best500FindAllDegree1_wl500,0.770337,0.195020,7592,83,82,57,0.673,0.734,0.595,0.768,0.153,0.816,0.253,0.987



 best500FindAllDegree1_wl500 classes < 0.7 sensitivity


id
0                                       ACC
8                                      BRCA
10                           Brain_Amygdala
11     Brain_Anterior_cingulate_cortex_BA24
12              Brain_Caudate_basal_ganglia
16                 Brain_Frontal_Cortex_BA9
17                        Brain_Hippocampus
18                       Brain_Hypothalamus
19    Brain_Nucleus_accumbens_basal_ganglia
20              Brain_Putamen_basal_ganglia
23                    Breast_Mammary_Tissue
26                                     COAD
31                         Colon_Transverse
33                                     ESCA
34      Esophagus_Gastroesophageal_Junction
38                                     HNSC
41                                     KICH
43                                     KIRP
45                                      LGG
47                                     LUAD
55                                       OV
64                                     SARC
66                           

id,mean_sensitivity,std_sensitivity,numGenes,numTypes,numDegree1,numAboveThreshold,LUAD,LUSC,COAD,READ,ESCA,LIHC,STAD,Whole_Blood
best20FindAllDegree1_wl5,0.708614,0.216666,313,83,46,45,0.443,0.502,0.456,0.786,0.162,0.735,0.280,0.987
best500FindAllDegree1_wl500,0.770337,0.195020,7592,83,82,57,0.673,0.734,0.595,0.768,0.153,0.816,0.253,0.987


In [7]:
best500FindAllDegree1_wl500DictPath = "/private/groups/kimlab/aedavids/deconvolution/1vsAll-~gender_category/best500FindAllDegree1_wl500/training/best500FindAllDegree1_wl500.sh.out/upsetPlot.out/best500_findAllDegree1_wl500.intersection.dict"
best500FindAllDegree1_wl500Dict = loadDictionary(best500FindAllDegree1_wl500DictPath)
best500FindAllDegree1_wl500Degree1 = findIntersectionsWithDegree(best500FindAllDegree1_wl500Dict, degree=1)
degree1CategoriesSet = findAllCategories(best500FindAllDegree1_wl500Degree1)
degree1GenesSet = findAllGenes(best500FindAllDegree1_wl500Degree1)
print(f'len(degree1CategoriesSet) : {len(degree1CategoriesSet)}')
print(f'len(degree1GenesSet) : {len(degree1GenesSet)}')

len(degree1CategoriesSet) : 82
len(degree1GenesSet) : 3999


In [8]:
statsCols = [ 'mean_sensitivity', 'std_sensitivity', 'numGenes',
               'numTypes', 'numDegree1', 'numAboveThreshold']
categoryCols = ~bestMetricsAllDegree1DF.columns.isin(statsCols)
runName = 'best20FindAllDegree1_wl5'
allCategories = bestMetricsAllDegree1DF.loc[runName, categoryCols].index
print(f' len(allCategoriesSet) : {len(allCategories)}')

 len(allCategoriesSet) : 83


In [9]:
print(f'best500FindAllDegree1_wl500 is missing { set(allCategories) - degree1CategoriesSet}')

best500FindAllDegree1_wl500 is missing {'Brain_Anterior_cingulate_cortex_BA24'}


# Explore BestCuratedGeneConfig
**<span style="color:red">TODO update the discription</span>** 

select top n genes sorted by base mean from the degree1 intersection. Uses intersection dictionary from best500FindAllDegree1_wl500. These are the degree 1 genes from the best top 1000 genes

git commit id ce467ff  

no degree1 for ('Brain_Anterior_cingulate_cortex_BA24',)  what was the top gene we returned?



I think gene returned my just be the first one alphabetically. Should we just return an empty dataframe?

countData="${rootDir}/groupbyGeneTrainingSets/GTEx_TCGA_TrainGroupby.csv"

```
/private/groups/kimlab/aedavids/deconvolution/1vsAll-~gender_category/best1CuratedDegree1/training/best1CuratedDegree1.sh.out/GTEx_TCGA-design-tilda_gender_category-padj-0001-lfc-20-n-1

$ cat Brain_Anterior_cingulate_cortex_BA24_vs_all.results 
name,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
AC022149.1,167.301764998255,-23.2955674454774,0.623116164760697,-37.3855931893916,6.699001102065111e-306,2.78644950840398e-301
```

In [10]:
def evaluateBestGTEx_TCGA():
    bestGTEx_TCGAResultsDir = [  
        'best1GTEx_TCGA',
        'best2GTEx_TCGA',
        'best3GTEx_TCGA',
        'best5GTEx_TCGA',
        'best10GTEx_TCGA',   
    ]
    
    threshold=0.7
    stageName =  "best10GTEx_TCGA"
    bestMetricsGTEx_TCGA1DF, bestGTEx_TCGABellowThresholdDF = evaluate(
                                                        root=root,
                                                        outDir=outDir,
                                                        resultsDirs=bestGTEx_TCGAResultsDir, 
                                                        outFilePrefix=stageName,
                                                        metric='sensitivity',
                                                        stageName=stageName,
                                                        threshold=threshold,
                                                        )
    

print(f'##### use bestNGTEx_TCGA run results as a base line to compare bestCuratedDegree1 results\n\n')
evaluateBestGTEx_TCGA()

##### use bestNGTEx_TCGA run results as a base line to compare bestCuratedDegree1 results



saving : /private/groups/kimlab/aedavids/deconvolution/1vsAll-~gender_category/hyperParameter/hyperparameterTunningResults4.out/best10GTEx_TCGA.sensitivity.0.7.csv

saving : /private/groups/kimlab/aedavids/deconvolution/1vsAll-~gender_category/hyperParameter/hyperparameterTunningResults4.out/best10GTEx_TCGA.sensitivity.bellow.0.7.csv


id,mean_sensitivity,std_sensitivity,numGenes,numTypes,numDegree1,numAboveThreshold,LUAD,LUSC,COAD,READ,ESCA,LIHC,STAD,Whole_Blood
best1GTEx_TCGA,0.233072,0.296911,19,83,7,10,0.117,0.389,0.025,0.196,0.036,0.753,0.249,0.000
best2GTEx_TCGA,0.361193,0.323777,30,83,10,16,0.210,0.369,0.177,0.214,0.171,0.785,0.382,0.993
best3GTEx_TCGA,0.491145,0.294545,46,83,17,23,0.430,0.339,0.177,0.375,0.387,0.879,0.418,0.982
best5GTEx_TCGA,0.571506,0.274796,70,83,16,32,0.220,0.435,0.222,0.464,0.333,0.803,0.347,0.978
best10GTEx_TCGA,0.675398,0.230238,136,83,30,42,0.453,0.455,0.500,0.536,0.270,0.789,0.347,0.987



 best10GTEx_TCGA classes < 0.7 sensitivity


id
0                                       ACC
2                  Adipose_Visceral_Omentum
5                           Artery_Coronary
7                                      BLCA
8                                      BRCA
9                                   Bladder
10                           Brain_Amygdala
11     Brain_Anterior_cingulate_cortex_BA24
12              Brain_Caudate_basal_ganglia
14                         Brain_Cerebellum
16                 Brain_Frontal_Cortex_BA9
17                        Brain_Hippocampus
18                       Brain_Hypothalamus
19    Brain_Nucleus_accumbens_basal_ganglia
20              Brain_Putamen_basal_ganglia
22                   Brain_Substantia_nigra
23                    Breast_Mammary_Tissue
24                                     CESC
25                                     CHOL
26                                     COAD
31                         Colon_Transverse
33                                     ESCA
34      Esophagus_Gastroesoph

In [11]:
def evaluateBestCuratedDegree1(resultsDir : list[str], metric='sensitivity'):

    
    threshold=0.7
    stageName = "best10CuratedDegree1"
    bestCuratedDegree1Metrics1DF, bestCuratedDegree1BellowThresholdDF = evaluate(
                                                        root=root,
                                                        outDir=outDir,
                                                        resultsDirs=resultsDir, 
                                                        outFilePrefix=stageName,
                                                        metric=metric,
                                                        stageName=stageName,
                                                        threshold=threshold,
                                                        verbose=False,
                                                        )
bestCurratedDegree1Dict = {
    "best1CuratedDegree1" : [  
        "best1CuratedDegree1", 
        "best1CuratedDegree1_6457b56", # bug we did not sort in decending order
        "best1CuratedDegree1.noArtifical", # does not artifically enrich
        "best1CuratedDegree1_ce467ff" ],
    
    "best2CuratedDegree1" : [ 
        "best2CuratedDegree1",      
        "best2CuratedDegree1_6457b56",  # bug we did not sort in decending order        
        "best2CuratedDegree1.noArtifical", # does not artifically enrich
        "best2CuratedDegree1_ce467ff"],   
    
    "best3CuratedDegree1" : [ 
        "best3CuratedDegree1",
        "best3CuratedDegree1_6457b56", # bug we did not sort in decending order
        "best3CuratedDegree1_de50c43", 
        "best3CuratedDegree1.noArtifical", # does not artifically enrich        
        "best3CuratedDegree1_ce467ff"],  
    
    "best5CuratedDegree1" : [ 
        "best5CuratedDegree1", 
        "best5CuratedDegree1_6457b56",  # bug we did not sort in decending order
        "best5CuratedDegree1.noArtifical", # does not artifically enrich        
        "best5CuratedDegree1_ce467ff"],   
    
    "best10CuratedDegree1" : [ 
        "best10CuratedDegree1",
        "best10CuratedDegree1_6457b56", # bug we did not sort in decending order
        "best10CuratedDegree1.noArtifical", # does not artifically enrich        
        "best10CuratedDegree1_ce467ff"],   
 }

for name,resultsDir in bestCurratedDegree1Dict.items() :
    print(f'\n###### {name}')
    evaluateBestCuratedDegree1(resultsDir)

warning="Notice best10CuratedDegree1_ce467ff has 716 genes, we expected 830." \
        + "\nIs this because Brain_Anterior_cingulate_cortex_BA24 added junk?"
print(f'\n!!!!! {warning}')


###### best1CuratedDegree1

saving : /private/groups/kimlab/aedavids/deconvolution/1vsAll-~gender_category/hyperParameter/hyperparameterTunningResults4.out/best10CuratedDegree1.sensitivity.0.7.csv

saving : /private/groups/kimlab/aedavids/deconvolution/1vsAll-~gender_category/hyperParameter/hyperparameterTunningResults4.out/best10CuratedDegree1.sensitivity.bellow.0.7.csv


id,mean_sensitivity,std_sensitivity,numGenes,numTypes,numDegree1,numAboveThreshold,LUAD,LUSC,COAD,READ,ESCA,LIHC,STAD,Whole_Blood
best1CuratedDegree1,0.486398,0.359127,83,83,83,30,0.246,0.203,0.304,0.607,0.486,0.789,0.431,0.989
best1CuratedDegree1_6457b56,0.449627,0.343284,83,83,83,28,0.350,0.289,0.418,0.518,0.775,0.910,0.329,0.000
best1CuratedDegree1.noArtifical,0.615627,0.254956,82,83,82,34,0.307,0.369,0.228,0.411,0.072,0.610,0.289,0.987
best1CuratedDegree1_ce467ff,0.616494,0.253984,83,83,83,34,0.317,0.362,0.253,0.411,0.072,0.596,0.311,0.987



 best10CuratedDegree1 classes < 0.7 sensitivity


Series([], Name: category, dtype: object)


###### best2CuratedDegree1

saving : /private/groups/kimlab/aedavids/deconvolution/1vsAll-~gender_category/hyperParameter/hyperparameterTunningResults4.out/best10CuratedDegree1.sensitivity.0.7.csv

saving : /private/groups/kimlab/aedavids/deconvolution/1vsAll-~gender_category/hyperParameter/hyperparameterTunningResults4.out/best10CuratedDegree1.sensitivity.bellow.0.7.csv


...


###### best10CuratedDegree1

saving : /private/groups/kimlab/aedavids/deconvolution/1vsAll-~gender_category/hyperParameter/hyperparameterTunningResults4.out/best10CuratedDegree1.sensitivity.0.7.csv

saving : /private/groups/kimlab/aedavids/deconvolution/1vsAll-~gender_category/hyperParameter/hyperparameterTunningResults4.out/best10CuratedDegree1.sensitivity.bellow.0.7.csv


id,mean_sensitivity,std_sensitivity,numGenes,numTypes,numDegree1,numAboveThreshold,LUAD,LUSC,COAD,READ,ESCA,LIHC,STAD,Whole_Blood
best10CuratedDegree1,0.786482,0.206089,716,83,83,62,0.485,0.595,0.627,0.679,0.396,0.906,0.271,0.987
best10CuratedDegree1_6457b56,0.747024,0.196196,716,83,83,49,0.725,0.684,0.557,0.643,0.378,0.883,0.324,1.000
best10CuratedDegree1.noArtifical,0.803410,0.194048,706,83,82,61,0.751,0.628,0.684,0.518,0.162,0.767,0.231,1.000
best10CuratedDegree1_ce467ff,0.808831,0.202235,716,83,83,63,0.841,0.691,0.652,0.607,0.369,0.874,0.409,1.000



 best10CuratedDegree1 classes < 0.7 sensitivity


id
8                                     BRCA
10                          Brain_Amygdala
11    Brain_Anterior_cingulate_cortex_BA24
12             Brain_Caudate_basal_ganglia
17                       Brain_Hippocampus
20             Brain_Putamen_basal_ganglia
22                  Brain_Substantia_nigra
23                   Breast_Mammary_Tissue
26                                    COAD
31                        Colon_Transverse
33                                    ESCA
34     Esophagus_Gastroesophageal_Junction
36                    Esophagus_Muscularis
47                                    LUAD
48                                    LUSC
57                                    PAAD
63                                    READ
64                                    SARC
66                                    STAD
69          Small_Intestine_Terminal_Ileum
81                                  Vagina
Name: category, dtype: object


!!!!! Notice best10CuratedDegree1_ce467ff has 716 genes, we expected 830.
Is this because Brain_Anterior_cingulate_cortex_BA24 added junk?


In [12]:
 evaluateBestCuratedDegree1(bestCurratedDegree1Dict["best10CuratedDegree1"], metric='specificity')


saving : /private/groups/kimlab/aedavids/deconvolution/1vsAll-~gender_category/hyperParameter/hyperparameterTunningResults4.out/best10CuratedDegree1.specificity.0.7.csv

saving : /private/groups/kimlab/aedavids/deconvolution/1vsAll-~gender_category/hyperParameter/hyperparameterTunningResults4.out/best10CuratedDegree1.specificity.bellow.0.7.csv


id,mean_specificity,std_specificity,numGenes,numTypes,numDegree1,numAboveThreshold,LUAD,LUSC,COAD,READ,ESCA,LIHC,STAD,Whole_Blood
best10CuratedDegree1,0.997542,0.003030,716,83,83,83,1.000,0.994,0.993,0.991,0.998,0.999,1.000,1.000
best10CuratedDegree1_6457b56,0.997229,0.005225,716,83,83,83,0.998,0.994,0.998,0.996,0.999,0.999,1.000,0.996
best10CuratedDegree1.noArtifical,0.997699,0.003529,706,83,82,83,0.999,0.996,0.981,0.994,1.000,1.000,1.000,0.999
best10CuratedDegree1_ce467ff,0.997855,0.002922,716,83,83,83,0.998,0.995,0.995,0.996,0.999,1.000,0.999,0.999



 best10CuratedDegree1 classes < 0.7 specificity


Series([], Name: category, dtype: object)

# Why are best10CuratedDegree1_ce467ff and best10CuratedDegree1 so different?

In [13]:
def aedwip(resultsDir, verbose:bool=False) :
    threshold=0.7
    stageName = "best10CuratedDegree1"
    bestCuratedDegree1Metrics1DF, bestCuratedDegree1BellowThresholdDF = evaluate(
                                                        root=root,
                                                        outDir=outDir,
                                                        resultsDirs=resultsDir, 
                                                        outFilePrefix=stageName,
                                                        metric='sensitivity',
                                                        stageName=stageName,
                                                        threshold=threshold,
                                                        verbose=verbose,
                                                        )
    return (bestCuratedDegree1Metrics1DF, bestCuratedDegree1BellowThresholdDF)
bestCuratedDegree1Metrics1DF, bestCuratedDegree1BellowThresholdDF =\
            aedwip(bestCurratedDegree1Dict['best10CuratedDegree1'], verbose=True)

path : /private/groups/kimlab/aedavids/deconvolution/1vsAll-~gender_category/best10CuratedDegree1

load best10CuratedDegree1 :
/private/groups/kimlab/aedavids/deconvolution/1vsAll-~gender_category/best10CuratedDegree1/training/best10CuratedDegree1.sh.out/metrics/metricsRounded.csv

load
/private/groups/kimlab/aedavids/deconvolution/1vsAll-~gender_category/best10CuratedDegree1/training/best10CuratedDegree1.sh.out/upsetPlot.out/best10_from_best500FindAllDegree1_wl500.intersection.dict

best10CuratedDegree1 types without degree 1 intersections: 
 set()
path : /private/groups/kimlab/aedavids/deconvolution/1vsAll-~gender_category/best10CuratedDegree1_6457b56

load best10CuratedDegree1_6457b56 :
/private/groups/kimlab/aedavids/deconvolution/1vsAll-~gender_category/best10CuratedDegree1_6457b56/training/best10CuratedDegree1.sh.out/metrics/metricsRounded.csv

load
/private/groups/kimlab/aedavids/deconvolution/1vsAll-~gender_category/best10CuratedDegree1_6457b56/training/best10CuratedDegree1.sh.

id,mean_sensitivity,std_sensitivity,numGenes,numTypes,numDegree1,numAboveThreshold,LUAD,LUSC,COAD,READ,ESCA,LIHC,STAD,Whole_Blood
best10CuratedDegree1,0.786482,0.206089,716,83,83,62,0.485,0.595,0.627,0.679,0.396,0.906,0.271,0.987
best10CuratedDegree1_6457b56,0.747024,0.196196,716,83,83,49,0.725,0.684,0.557,0.643,0.378,0.883,0.324,1.000
best10CuratedDegree1.noArtifical,0.803410,0.194048,706,83,82,61,0.751,0.628,0.684,0.518,0.162,0.767,0.231,1.000
best10CuratedDegree1_ce467ff,0.808831,0.202235,716,83,83,63,0.841,0.691,0.652,0.607,0.369,0.874,0.409,1.000



 best10CuratedDegree1 classes < 0.7 sensitivity


id
8                                     BRCA
10                          Brain_Amygdala
11    Brain_Anterior_cingulate_cortex_BA24
12             Brain_Caudate_basal_ganglia
17                       Brain_Hippocampus
20             Brain_Putamen_basal_ganglia
22                  Brain_Substantia_nigra
23                   Breast_Mammary_Tissue
26                                    COAD
31                        Colon_Transverse
33                                    ESCA
34     Esophagus_Gastroesophageal_Junction
36                    Esophagus_Muscularis
47                                    LUAD
48                                    LUSC
57                                    PAAD
63                                    READ
64                                    SARC
66                                    STAD
69          Small_Intestine_Terminal_Ileum
81                                  Vagina
Name: category, dtype: object

In [14]:
# load hte intersection dict , get all the genes . set diff/
best10CuratedDegree1_ce467ffDictPath = "/private/groups/kimlab/aedavids/deconvolution/1vsAll-~gender_category/best10CuratedDegree1_ce467ff/training/best10CuratedDegree1.sh.out/upsetPlot.out/best10_from_best500FindAllDegree1_wl500.intersection.dict"
best10CuratedDegree1_ce467ffDict = loadDictionary(best10CuratedDegree1_ce467ffDictPath)
ce467ffLUADGenes = best10CuratedDegree1_ce467ffDict[('LUAD',)]
ce467ffLUADGenes

['AC011944.1',
 'ATP13A4-AS1',
 'AC090004.1',
 'GXYLT1P3',
 'AC126323.6',
 'AC111149.2',
 'AL365181.4',
 '(TCCAC)n',
 'BPIFA2',
 'IGHJ5']

In [15]:
best10CuratedDegree1DictPath = "/private/groups/kimlab/aedavids/deconvolution/1vsAll-~gender_category/best10CuratedDegree1/training/best10CuratedDegree1.sh.out/upsetPlot.out/best10_from_best500FindAllDegree1_wl500.intersection.dict"
best10CuratedDegree1Dict = loadDictionary(best10CuratedDegree1_ce467ffDictPath)
LUADGenes = best10CuratedDegree1Dict[('LUAD',)]
LUADGenes

['AC011944.1',
 'ATP13A4-AS1',
 'AC090004.1',
 'GXYLT1P3',
 'AC126323.6',
 'AC111149.2',
 'AL365181.4',
 '(TCCAC)n',
 'BPIFA2',
 'IGHJ5']

In [16]:
ce467ffAllGenes = findAllGenes(best10CuratedDegree1_ce467ffDict)
best10AllGenes = findAllGenes(best10CuratedDegree1Dict)

print(f'ce467ffAllGenes - best10AllGenes : {set(ce467ffAllGenes) - set(best10AllGenes)}')
print()
print(f'best10AllGenes - ce467ffAllGenes : {set(best10AllGenes) - set(ce467ffAllGenes)}')

ce467ffAllGenes - best10AllGenes : set()

best10AllGenes - ce467ffAllGenes : set()
